## Reading real profile jsons into df

In [261]:
import json
import os
import glob
import pandas as pd

json_dir = 'data/realprofile'
json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
realdf = pd.concat(dfs)
realdf.to_csv('uncleaned_real.csv', header=True, index=False)

## Reading scam profile jsons into df

In [262]:
import json
import os
import glob
import pandas as pd



json_dir = 'data/scamprofile'
json_pattern = os.path.join(json_dir, '*.json')
file_list = glob.glob(json_pattern)

dfs = []
for file in file_list:
    with open(file) as f:
        json_data = pd.json_normalize(json.loads(f.read()))
        json_data['site'] = file.rsplit("/", 1)[-1]
    dfs.append(json_data)
scamdf = pd.concat(dfs)
scamdf.to_csv('uncleaned_scam.csv', header=True, index=False)

In [288]:
scamdf = pd.read_csv("uncleaned_scam.csv")
scamdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4342 entries, 0 to 4341
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   year_reported   4342 non-null   int64 
 1   month_reported  4342 non-null   int64 
 2   username        4342 non-null   object
 3   name            4342 non-null   object
 4   age             4341 non-null   object
 5   occupation      4335 non-null   object
 6   status          4339 non-null   object
 7   inet            4341 non-null   object
 8   description     4255 non-null   object
 9   email           4336 non-null   object
 10  tags            4342 non-null   object
 11  gender          4342 non-null   object
 12  images          4342 non-null   object
 13  site            4342 non-null   object
 14  phone           190 non-null    object
dtypes: int64(2), object(13)
memory usage: 509.0+ KB


In [ ]:
scamdf.head()

# Scam df data cleaning 
- Select only relevant columns: username, age, occupation, status, gender, description
- keep only the 4255 rows with non null description
- For description column, 
    - remove the \n at the beginning of the sentence
    - remove punctuations
    - remove stop words
    - Use TF-IDF vectorizer to encode the descriptions (actually based on the research paper, they trained an SVM       algorithm (linear kernel) as implemented in LibShortText, an open-source software package for short-         text classification and analysis)
    - Text vectorization: https://www.oreilly.com/library/view/applied-text-analysis/9781491963036/ch04.html
    
    

In [290]:
pd.set_option("display.max_colwidth", None)

In [291]:
#select relevant columns
scamdf1 = scamdf[["username","age","occupation","status","gender","description"]].copy()
len(scamdf1)

4342

In [292]:
#drop all rows with no descriptions - 87 rows dropped
scamdf1 = scamdf1[scamdf1["description"].notna()]
len(scamdf1)

4255

In [293]:
#remove punctuation
import re
def get_new_sentence(x):
    return re.sub(r'[^\w\s]', '', x)

scamdf1["description"] = scamdf1["description"].apply(lambda x: get_new_sentence(x))

#clean age column
def get_age(x):
    x = str(x)
    if 'or' in x:
        return x.split()[0]
    else:
        return x

scamdf1["age"] = scamdf1["age"].apply(lambda x: get_age(x))

In [294]:
#remove stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stop_words(description):
    
    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(description)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return ' '.join(filtered_sentence)

#create a new column to store the filtered sentences
scamdf1["filtered sentence"] = scamdf1["description"].apply(lambda x: remove_stop_words(x))

In [295]:
scamdf1.head()

,username,age,occupation,status,gender,description,filtered sentence
0,000love,33,student,single,female,\ni am cool simple easy going easy to please if treated with resp ect i am cool simple easy going easy to please if treated with resp ecti am cool simple easy going easy to please if treated with resp ect,cool simple easy going easy please treated resp ect cool simple easy going easy please treated resp ecti cool simple easy going easy please treated resp ect
1,05johnsmith,57,contractor,widowed,male,\ni am kind honest love and caring man my hobby are music and sport i believe in god,kind honest love caring man hobby music sport believe god
2,1234,57,business,widowed,male,\nAs you know that its myriads that someone has lots of interested women in here but its my choice to choose onethe one my heart goes forsomeone i can lean onto give me the love i have always wanted to haveSo tell me about yourselfwhat you do and everything you need to tell me about yourselfand lets see how it goes from hereBut anywayHeres a little info about me I like to go out to dinner go dancinggo to the movies workout travel see new things I enjoy the simple things like the ocean cuddling at home watching a movie with someone special Im a extremely romantic passionate and affectionate woman,As know myriads someone lots interested women choice choose onethe one heart goes forsomeone lean onto give love always wanted haveSo tell yourselfwhat everything need tell yourselfand lets see goes hereBut anywayHeres little info I like go dinner go dancinggo movies workout travel see new things I enjoy simple things like ocean cuddling home watching movie someone special Im extremely romantic passionate affectionate woman
3,231411,53,Building Contractor,single,male,\nI am looking for a woman whom I think is handsome sexy and smart A perfect simple woman one who treats me well and who makes me laugh she is not afraid to grab my hand and steal a kiss in public or to just shoot a knowing stare across the room that tells me she is admiring me from afarAt this stage in my life I am focused on finding you I have a great deal to offer and will always provide a warm heart and welcoming smile I am optimistic youre out there and perhaps ifwhen you read this far in my profile it may resonate with you\nBe smart and contact me directly henshawmark is my ID and to figure the network you ddd the first alphabet of the following word in capital Yoke Apple Hot Orange Orange Let see if you solve it by writing me Had to sneak this in lol,I looking woman I think handsome sexy smart A perfect simple woman one treats well makes laugh afraid grab hand steal kiss public shoot knowing stare across room tells admiring afarAt stage life I focused finding I great deal offer always provide warm heart welcoming smile I optimistic youre perhaps ifwhen read far profile may resonate Be smart contact directly henshawmark ID figure network ddd first alphabet following word capital Yoke Apple Hot Orange Orange Let see solve writing Had sneak lol
4,3234714190Txtme,33,self employed,single,female,\nJust a note I used to be able to do a name search for people using the Meet Me feature I can no longer do that so please send a message instead or at least hit Favorite so I can find you\nI have a really good sense of humor quickthinking wit Very kindhearted and sometimes maybe too logical I love to travel have game nights bar b ques with friends and family I only drink socially have a good group of family and friends who when we do get together laugh a lot I love being active and in nature ride bikes exploring new places I love old school rock and some older country and wherever I am if I know the song playing I may accidentally start singing out loud in front of other people hopefully youll join me I cry every time I hear The National Anthem Im a simple girl and try and be as stressfree as possible Ive never been more selfaware and selfconfident in my life Very happy and excited about my future in so many wa

In [296]:
scamdf1["y"] = 1 #1 for scam profiles
scamdf1.head()

,username,age,occupation,status,gender,description,filtered sentence,y
0,000love,33,student,single,female,\ni am cool simple easy going easy to please if treated with resp ect i am cool simple easy going easy to please if treated with resp ecti am cool simple easy going easy to please if treated with resp ect,cool simple easy going easy please treated resp ect cool simple easy going easy please treated resp ecti cool simple easy going easy please treated resp ect,1
1,05johnsmith,57,contractor,widowed,male,\ni am kind honest love and caring man my hobby are music and sport i believe in god,kind honest love caring man hobby music sport believe god,1
2,1234,57,business,widowed,male,\nAs you know that its myriads that someone has lots of interested women in here but its my choice to choose onethe one my heart goes forsomeone i can lean onto give me the love i have always wanted to haveSo tell me about yourselfwhat you do and everything you need to tell me about yourselfand lets see how it goes from hereBut anywayHeres a little info about me I like to go out to dinner go dancinggo to the movies workout travel see new things I enjoy the simple things like the ocean cuddling at home watching a movie with someone special Im a extremely romantic passionate and affectionate woman,As know myriads someone lots interested women choice choose onethe one heart goes forsomeone lean onto give love always wanted haveSo tell yourselfwhat everything need tell yourselfand lets see goes hereBut anywayHeres little info I like go dinner go dancinggo movies workout travel see new things I enjoy simple things like ocean cuddling home watching movie someone special Im extremely romantic passionate affectionate woman,1
3,231411,53,Building Contractor,single,male,\nI am looking for a woman whom I think is handsome sexy and smart A perfect simple woman one who treats me well and who makes me laugh she is not afraid to grab my hand and steal a kiss in public or to just shoot a knowing stare across the room that tells me she is admiring me from afarAt this stage in my life I am focused on finding you I have a great deal to offer and will always provide a warm heart and welcoming smile I am optimistic youre out there and perhaps ifwhen you read this far in my profile it may resonate with you\nBe smart and contact me directly henshawmark is my ID and to figure the network you ddd the first alphabet of the following word in capital Yoke Apple Hot Orange Orange Let see if you solve it by writing me Had to sneak this in lol,I looking woman I think handsome sexy smart A perfect simple woman one treats well makes laugh afraid grab hand steal kiss public shoot knowing stare across room tells admiring afarAt stage life I focused finding I great deal offer always provide warm heart welcoming smile I optimistic youre perhaps ifwhen read far profile may resonate Be smart contact directly henshawmark ID figure network ddd first alphabet following word capital Yoke Apple Hot Orange Orange Let see solve writing Had sneak lol,1
4,3234714190Txtme,33,self employed,single,female,\nJust a note I used to be able to do a name search for people using the Meet Me feature I can no longer do that so please send a message instead or at least hit Favorite so I can find you\nI have a really good sense of humor quickthinking wit Very kindhearted and sometimes maybe too logical I love to travel have game nights bar b ques with friends and family I only drink socially have a good group of family and friends who when we do get together laugh a lot I love being active and in nature ride bikes exploring new places I love old school rock and some older country and wherever I am if I know the song playing I may accidentally start singing out loud in front of other people hopefully youll join me I cry every time I hear The National Anthem Im a simple girl and try and be as stressfree as possible Ive never been more selfaware and selfconfident in my life Very happy and excited about my future in 

In [297]:
scamdf1.drop(columns = "description", inplace = True)

In [298]:
scamdf1.head()

,username,age,occupation,status,gender,filtered sentence,y
0,000love,33,student,single,female,cool simple easy going easy please treated resp ect cool simple easy going easy please treated resp ecti cool simple easy going easy please treated resp ect,1
1,05johnsmith,57,contractor,widowed,male,kind honest love caring man hobby music sport believe god,1
2,1234,57,business,widowed,male,As know myriads someone lots interested women choice choose onethe one heart goes forsomeone lean onto give love always wanted haveSo tell yourselfwhat everything need tell yourselfand lets see goes hereBut anywayHeres little info I like go dinner go dancinggo movies workout travel see new things I enjoy simple things like ocean cuddling home watching movie someone special Im extremely romantic passionate affectionate woman,1
3,231411,53,Building Contractor,single,male,I looking woman I think handsome sexy smart A perfect simple woman one treats well makes laugh afraid grab hand steal kiss public shoot knowing stare across room tells admiring afarAt stage life I focused finding I great deal offer always provide warm heart welcoming smile I optimistic youre perhaps ifwhen read far profile may resonate Be smart contact directly henshawmark ID figure network ddd first alphabet following word capital Yoke Apple Hot Orange Orange Let see solve writing Had sneak lol,1
4,3234714190Txtme,33,self employed,single,female,Just note I used able name search people using Meet Me feature I longer please send message instead least hit Favorite I find I really good sense humor quickthinking wit Very kindhearted sometimes maybe logical I love travel game nights bar b ques friends family I drink socially good group family friends get together laugh lot I love active nature ride bikes exploring new places I love old school rock older country wherever I I know song playing I may accidentally start singing loud front people hopefully youll join I cry every time I hear The National Anthem Im simple girl try stressfree possible Ive never selfaware selfconfident life Very happy excited future many ways Work love possibilities endless I love love great smile someone A good kisser must,1


In [299]:
#SCAMDF checking for empty values
scamdf1[scamdf1["filtered sentence"] == '']

,username,age,occupation,status,gender,filtered sentence,y
20,adalhardbartholo,42,Oil and Gas consultant,single,male,,1
26,adamsandler,53,doctor,single,male,,1
30,adamslilly,51,laywer,widowed,female,,1
31,adamsmorgan,46,manufacturing,single,male,,1
39,Adriana200,30,self employed,single,female,,1
...,...,...,...,...,...,...,...
4297,wilsonmorgan,48,engineer,widowed,male,,1
4314,wyanebruce,42,–,–,male,,1
4319,w_paul67,38,–,–,male,,1
4329,yolly36,36,military,single,female,,1


In [300]:
scamdf1 = scamdf1[scamdf1["filtered sentence"] != ''].copy()

In [301]:
#remove rows that are not in english 
scamdf1['Language'] = scamdf1['filtered sentence'].apply(detect)
scamdf1 = scamdf1[scamdf1['Language']=='en'].copy()

In [302]:
scamdf1

,username,age,occupation,status,gender,filtered sentence,y,Language
0,000love,33,student,single,female,cool simple easy going easy please treated resp ect cool simple easy going easy please treated resp ecti cool simple easy going easy please treated resp ect,1,en
1,05johnsmith,57,contractor,widowed,male,kind honest love caring man hobby music sport believe god,1,en
2,1234,57,business,widowed,male,As know myriads someone lots interested women choice choose onethe one heart goes forsomeone lean onto give love always wanted haveSo tell yourselfwhat everything need tell yourselfand lets see goes hereBut anywayHeres little info I like go dinner go dancinggo movies workout travel see new things I enjoy simple things like ocean cuddling home watching movie someone special Im extremely romantic passionate affectionate woman,1,en
3,231411,53,Building Contractor,single,male,I looking woman I think handsome sexy smart A perfect simple woman one treats well makes laugh afraid grab hand steal kiss public shoot knowing stare across room tells admiring afarAt stage life I focused finding I great deal offer always provide warm heart welcoming smile I optimistic youre perhaps ifwhen read far profile may resonate Be smart contact directly henshawmark ID figure network ddd first alphabet following word capital Yoke Apple Hot Orange Orange Let see solve writing Had sneak lol,1,en
4,3234714190Txtme,33,self employed,single,female,Just note I used able name search people using Meet Me feature I longer please send message instead least hit Favorite I find I really good sense humor quickthinking wit Very kindhearted sometimes maybe logical I love travel game nights bar b ques friends family I drink socially good group family friends get together laugh lot I love active nature ride bikes exploring new places I love old school rock older country wherever I I know song playing I may accidentally start singing loud front people hopefully youll join I cry every time I hear The National Anthem Im simple girl try stressfree possible Ive never selfaware selfconfident life Very happy excited future many ways Work love possibilities endless I love love great smile someone A good kisser must,1,en
...,...,...,...,...,...,...,...,...
4336,zarifa2,24,student,single,female,I never know I write part lameSo Im ordinary nerd high school student last year I study English Polish History subjects connected interests generally Im interersted much things Then I hope gratuade study something realated university Maybe museology I dont know I plans I spend free time mostly writing letters drawing writing stories reading books watching tv series playing games Im meet penpals common interests Addicted tea tumblr music Im typical introvert INFJ check type really describes well sometimes nerdy af Shy first thats I rarely write first sorry need courage,1,en
4338,zerenab,31,self employment,single,female,I serious mature woman good valueslearned home Family first I respectful honest neat organized hardworking I wish find good person raise pretty family I like keep word I expect others I need sincere gentleman good manners intentions respectful detailing somebody likes standout work much life A man good feelings believes honesty perfect man,1,en
4339,Ziah,25,Flower designer course,single,female,I love VOLLEYBALL I love MUSIC I love PICTURES I love DANCE I love PINK I LOVE SMILE I LOVE play SPORTS Im tight chick wan na know morejust ask,1,en
4340,zikki100,36,nurse,single,female,Im good woman looking good man One best friend soulmate I seranade love song written run candle lit bathcook home cooked dinner Then settle snuggle couch romantic movie,1,en


In [303]:
# drop language column
scamdf1.drop(columns='Language', inplace=True)

In [310]:
# removing occupation value count ==1 
scamdf1['occupation'].value_counts()

temp=scamdf1['occupation'].tolist()
scamdf1 = scamdf1[scamdf1['occupation'].apply(lambda x: temp.count(x)>1)].copy()


In [311]:
scamdf1['occupation'].value_counts()

 military              376
 self employed         272
 student               263
                       241
 engineer              210
                      ... 
 civil servant           2
 sales  marketing        2
 Hair stylist            2
 Project Management      2
 Hair dresser            2
Name: occupation, Length: 222, dtype: int64

In [312]:
tempstatus=scamdf1['status'].tolist()
scamdf1=scamdf1[scamdf1['status'].apply(lambda x: tempstatus.count(x)>1)].copy()

In [313]:
scamdf1['status'].value_counts()

 single                1517
 widowed                838
 divorced               415
                        125
 separated               22
 widower                 11
 widow                   11
 Widowed                  9
 Divorced                 8
 divorce                  2
 single or divorced       2
Name: status, dtype: int64

In [315]:
scamdf1=scamdf1.dropna()

In [316]:
scamdf1["age"] = scamdf1["age"].apply(lambda x: get_new_sentence(x))
scamdf1["occupation"] = scamdf1["occupation"].apply(lambda x: get_new_sentence(x))
scamdf1["status"] = scamdf1["status"].apply(lambda x: get_new_sentence(x))
scamdf1["gender"] = scamdf1["gender"].apply(lambda x: get_new_sentence(x))

In [317]:
scamdf1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2960 entries, 0 to 4341
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   username           2960 non-null   object
 1   age                2960 non-null   object
 2   occupation         2960 non-null   object
 3   status             2960 non-null   object
 4   gender             2960 non-null   object
 5   filtered sentence  2960 non-null   object
 6   y                  2960 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 185.0+ KB


In [318]:
# final 2960 rows
scamdf1.to_csv('scam_profile.csv', header=True, index=False)

# Real df data cleaning
- Select only relevant columns - username, gender, age, occupation, status, description
- For description, 
    - if description == '-':
        - description = ethnicity + children + orientation + religion + smoking + drinking + intent
    - remove punctuations 
    - check how many non english descriptions 

In [264]:
realdf = pd.read_csv("uncleaned_real.csv")

In [287]:
realdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5583 entries, 26 to 17722
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   username           5583 non-null   object
 1   age                5583 non-null   object
 2   occupation         5583 non-null   object
 3   status             5583 non-null   object
 4   gender             5583 non-null   object
 5   filtered sentence  5583 non-null   object
 6   y                  5583 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 348.9+ KB


In [265]:
realdf2 = realdf[realdf["description"] == "-"].copy()

In [266]:
realdf2["description"] = realdf2["ethnicity"] +' '+ realdf2["orientation"] +' '+ realdf2["religion"] +' '+ realdf2["smoking"] +' '+ realdf2["drinking"] +' '+ realdf2["intent"]
realdf2.head(10)


,gender,age,location,status,username,ethnicity,occupation,description,match_age,children,orientation,religion,smoking,drinking,intent,site
0,female,39 y.o.,"Johannesburg, South Africa",single,00,black,Frailcare nurse,black Straight Christian non-smoker social drinker Romance,from 36 to 50,1-2 living with me,Straight,Christian,non-smoker,social drinker,Romance,realprofile\real00.json
1,female,43 y.o.,"Manizales, Caldas, Colombia",separated,0065patricia,native american,Administradora,native american Bisexual Other non-smoker never Serious Relationship,from 19 to 86,more than 2 living elsewhere,Bisexual,Other,non-smoker,never,Serious Relationship,realprofile\real0065patricia.json
2,male,63 y.o.,"Lima, Perú",divorced,007,white,comercio,white Straight Christian non-smoker occasional drinker Friendship,from 19 to 86,1-2 living elsewhere,Straight,Christian,non-smoker,occasional drinker,Friendship,realprofile\real007.json
3,male,59 y.o.,"London, UK",divorced,0077654321,white,Developer,"white Straight Atheist non-smoker occasional drinker Fun, Friendship, Romance, Serious Relationship",from 48 to 65,1-2 living elsewhere,Straight,Atheist,non-smoker,occasional drinker,"Fun, Friendship, Romance, Serious Relationship",realprofile\real0077654321.json
4,male,30 y.o.,"Sucre, Bolivia",single,00alex00,hispanic,Guardia de seguridad,"hispanic Straight Other non-smoker never Fun, Friendship, Romance",from 19 to 86,don't want children,Straight,Other,non-smoker,never,"Fun, Friendship, Romance",realprofile\real00alex00.json
5,male,32 y.o.,"Colinas de Cerro Viento, Panamá",single,00lucero90,hispanic,Ingeniero naval,"hispanic Straight Christian social smoker social drinker Fun, Friendship, Romance",from 19 to 86,no children,Straight,Christian,social smoker,social drinker,"Fun, Friendship, Romance",realprofile\real00lucero90.json
7,male,52 y.o.,"Bucaramanga, Santander, Colombia",divorced,01Anselmo212,hispanic,Ingeniero,hispanic Straight Christian non-smoker never Serious Relationship,from 30 to 40,1-2 living elsewhere,Straight,Christian,non-smoker,never,Serious Relationship,realprofile\real01Anselmo212.json
8,male,49 y.o.,"Miami, Florida, EE. UU.",divorced,0279602,hispanic,Mantenimiento,hispanic Straight Other non-smoker never Serious Relationship,from 33 to 50,1-2 living elsewhere,Straight,Other,non-smoker,never,Serious Relationship,realprofile\real0279602.json
10,male,40 y.o.,"Chicago, IL, USA",single,03Robert,hispanic,-,"hispanic Straight Other light smoker social drinker Friendship, Romance, Serious Relationship",from 28 to 41,want children,Straight,Other,light smoker,social drinker,"Friendship, Romance, Serious Relationship",realprofile\real03Robert.json
11,male,49 y.o.,"Arequipa, Perú",single,06Wil09,hispanic,Docente,"hispanic Straight Christian non-smoker never Fun, Romance, Marriage",from 21 to 40,no children,Straight,Christian,non-smoker,never,"Fun, Romance, Marriage",realprofile\real06Wil09.json


In [267]:
#combine realdf2 with realdf3 
realdf3 = realdf[realdf["description"] != '-']
realdf4 = pd.concat([realdf3, realdf2])
len(realdf4)

17724

In [268]:

from langdetect import detect
realdf4['Language'] = realdf4['description'].apply(detect)


In [269]:
realdf4.head()

,gender,age,location,status,username,ethnicity,occupation,description,match_age,children,orientation,religion,smoking,drinking,intent,site,Language
6,male,58 y.o.,"San Salvador, El Salvador",divorced,0119162866,hispanic,Negocio,"Honesto. Trabajador, divertido. Me gusta la naturaleza, viajar, bailar.",from 30 to 42,no children,Straight,Spiritual,social smoker,never,Friendship,realprofile\real0119162866.json,es
9,male,38 y.o.,"Boaco, Nicaragua",single,02erick,hispanic,Decoraciones eventos privados,Soy un hombre sociable cariñoso y me gusta conoser personas y tener la oportunidad de establecer una relación en algún momento,from 24 to 39,want children,Straight,Christian,social smoker,occasional drinker,"Fun, Friendship, Romance, Serious Relationship",realprofile\real02erick.json,es
18,male,35 y.o.,"Bogota, Colombia",single,1047373361es,native american,Electrcista,Honesta honrrada y trabajadora y muy alegre amigable,from 30 to 50,no children,Straight,Other,non-smoker,never,"Serious Relationship, Marriage",realprofile\real1047373361es.json,es
22,female,40 y.o.,"Machala, Ecuador",single,12345bonita,black,-,Soy negra mido metro 150y soy gordita,from 19 to 86,more than 2 living with me,Bisexual,Christian,non-smoker,never,Serious Relationship,realprofile\real12345bonita.json,es
26,male,66 y.o.,"San Jose, CA, USA",single,123canwe,white,Retired,I am full of fire and friskier than hell!!!!,from 19 to 64,don't want children,Straight,Christian,non-smoker,never,"Fun, Friendship, Romance, Serious Relationship",realprofile\real123canwe.json,en


In [270]:
#removing rows that description is not english
realdf5 = realdf4[realdf4["Language"] == 'en'].copy()

In [271]:
#select relevant columns
realdf = realdf5[["username","age","occupation","status","gender","description"]].copy()

#remove 'y.o' from age
def get_age(x):
    return x.split('y')[0]
    
realdf["age"] = realdf["age"].apply(lambda x: get_age(x))

#remove punctuation
import re
def get_new_sentence(x):
    return re.sub(r'[^\w\s]', '', x).strip()

realdf["description"] = realdf["description"].apply(lambda x: get_new_sentence(x))

In [272]:
#remove stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stop_words(description):
    
    stop_words = set(stopwords.words('english'))

    word_tokens = word_tokenize(description)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    filtered_sentence = []
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return ' '.join(filtered_sentence)

#create a new column to store the filtered sentences
realdf["filtered sentence"] = realdf["description"].apply(lambda x: remove_stop_words(x))

In [273]:
realdf.head(20)

,username,age,occupation,status,gender,description,filtered sentence
26,123canwe,66,Retired,single,male,I am full of fire and friskier than hell,I full fire friskier hell
30,123WILFREDO,28,ESTUDIANTE,single,male,ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y MUY FUERTES EN LA MORAL,ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y MUY FUERTES EN LA MORAL
44,1907,48,Construction,single,male,laid back and down to earth with a good sense of humor,laid back earth good sense humor
59,1engineer,37,single engineer,single,male,I like to travel watch movies read books sports different kind of activities I have passion for life love being among people and with people,I like travel watch movies read books sports different kind activities I passion life love among people people
63,1SweetChick,56,-,single,female,Im looking to make friends Ive a great personality I love being happy and laughing and not being a negative person I like to spread the joy \r\nAnd \r\nmaybe a relationship at some point Have a great day,Im looking make friends Ive great personality I love happy laughing negative person I like spread joy And maybe relationship point Have great day
86,2sk00pz,49,Social Media Executive,single,male,Im a very friendly easygoing cheerful guy who loves to laugh and is very familyorientated as well as loyal and kind Id love to find my forever woman here,Im friendly easygoing cheerful guy loves laugh familyorientated well loyal kind Id love find forever woman
94,43hamon,46,Auto engineer,single,male,Hi wel i love to travel and love biking and the outdoors and like to keep busy and spend time with love ones,Hi wel love travel love biking outdoors like keep busy spend time love ones
104,5280TexasLady,48,medical field,divorced,female,If conversation rules the nation lets talkIf my appearance captivates you my conversation will keep youMy priority is friendship but my destiny is true loveI want more but why accept less,If conversation rules nation lets talkIf appearance captivates conversation keep youMy priority friendship destiny true loveI want accept less
105,52Jim52,70,Retired,divorced,male,Retired owner of aerospace consuloting firm,Retired owner aerospace consuloting firm
108,55GeorgeWhite,56,Welder Pipeline- Sell Real Estate,single,male,Sympathetic Cordial Gentleman with Joy good mood lively in a good mood with tenderness love and affection,Sympathetic Cordial Gentleman Joy good mood lively good mood tenderness love affection


In [274]:
realdf.drop(columns = "description", inplace = True)

In [275]:
realdf["y"] = 0
realdf.head()

,username,age,occupation,status,gender,filtered sentence,y
26,123canwe,66,Retired,single,male,I full fire friskier hell,0
30,123WILFREDO,28,ESTUDIANTE,single,male,ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y MUY FUERTES EN LA MORAL,0
44,1907,48,Construction,single,male,laid back earth good sense humor,0
59,1engineer,37,single engineer,single,male,I like travel watch movies read books sports different kind activities I passion life love among people people,0
63,1SweetChick,56,-,single,female,Im looking make friends Ive great personality I love happy laughing negative person I like spread joy And maybe relationship point Have great day,0


In [276]:
realtemp=realdf['occupation'].tolist()
realdf=realdf[realdf['occupation'].apply(lambda x: realtemp.count(x)>1)].copy()



In [277]:
realdf['occupation'].value_counts()

-                      4889
Estudiante              163
Retired                 151
Independiente           151
Comerciante             149
                       ... 
contador publico          2
Terapista                 2
Diseñadora Gráfica        2
Casa                      2
Mechanical engineer       2
Name: occupation, Length: 789, dtype: int64

In [278]:
realstatus=realdf['status'].tolist()
realdf=realdf[realdf['status'].apply(lambda x: realstatus.count(x)>1)].copy()

In [280]:
realdf['status'].value_counts()

single             6871
divorced           1745
separated          1202
widowed             265
in relationship     235
married             154
Name: status, dtype: int64

In [281]:
realdf[realdf['occupation']!='-']

,username,age,occupation,status,gender,filtered sentence,y
26,123canwe,66,Retired,single,male,I full fire friskier hell,0
30,123WILFREDO,28,ESTUDIANTE,single,male,ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y MUY FUERTES EN LA MORAL,0
44,1907,48,Construction,single,male,laid back earth good sense humor,0
105,52Jim52,70,Retired,divorced,male,Retired owner aerospace consuloting firm,0
146,Aaron90,28,Social worker,single,male,Hello aaron Vermont I vermont 2 years Im really fun person I like camping bone fire etc,0
...,...,...,...,...,...,...,...
17712,ZullynAzul2,25,Estudiante,single,female,white Straight Christian nonsmoker never Serious Relationship,0
17713,zunilda,66,docente,single,female,hispanic Straight Other nonsmoker never Friendship,0
17717,Zwadiemarie22,41,Probation Officer,separated,female,black Straight Christian nonsmoker occasional drinker Serious Relationship Marriage,0
17719,Zxxxx,26,Profesora,single,female,hispanic Straight Christian nonsmoker never Friendship Romance,0


In [282]:
# removing null occupation
realdf = realdf[realdf["occupation"] != '-'].copy()
realdf

,username,age,occupation,status,gender,filtered sentence,y
26,123canwe,66,Retired,single,male,I full fire friskier hell,0
30,123WILFREDO,28,ESTUDIANTE,single,male,ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y MUY FUERTES EN LA MORAL,0
44,1907,48,Construction,single,male,laid back earth good sense humor,0
105,52Jim52,70,Retired,divorced,male,Retired owner aerospace consuloting firm,0
146,Aaron90,28,Social worker,single,male,Hello aaron Vermont I vermont 2 years Im really fun person I like camping bone fire etc,0
...,...,...,...,...,...,...,...
17712,ZullynAzul2,25,Estudiante,single,female,white Straight Christian nonsmoker never Serious Relationship,0
17713,zunilda,66,docente,single,female,hispanic Straight Other nonsmoker never Friendship,0
17717,Zwadiemarie22,41,Probation Officer,separated,female,black Straight Christian nonsmoker occasional drinker Serious Relationship Marriage,0
17719,Zxxxx,26,Profesora,single,female,hispanic Straight Christian nonsmoker never Friendship Romance,0


In [283]:
realdf = realdf.dropna()

In [284]:
realdf["age"] = realdf["age"].apply(lambda x: get_new_sentence(x))
realdf["occupation"] = realdf["occupation"].apply(lambda x: get_new_sentence(x))
realdf["status"] = realdf["status"].apply(lambda x: get_new_sentence(x))
realdf["gender"] = realdf["gender"].apply(lambda x: get_new_sentence(x))

In [285]:
realdf

,username,age,occupation,status,gender,filtered sentence,y
26,123canwe,66,Retired,single,male,I full fire friskier hell,0
30,123WILFREDO,28,ESTUDIANTE,single,male,ME GUSTA CONOCER CHICAS BONDADOSAS CARIÑOSAS Y MUY FUERTES EN LA MORAL,0
44,1907,48,Construction,single,male,laid back earth good sense humor,0
105,52Jim52,70,Retired,divorced,male,Retired owner aerospace consuloting firm,0
146,Aaron90,28,Social worker,single,male,Hello aaron Vermont I vermont 2 years Im really fun person I like camping bone fire etc,0
...,...,...,...,...,...,...,...
17712,ZullynAzul2,25,Estudiante,single,female,white Straight Christian nonsmoker never Serious Relationship,0
17713,zunilda,66,docente,single,female,hispanic Straight Other nonsmoker never Friendship,0
17717,Zwadiemarie22,41,Probation Officer,separated,female,black Straight Christian nonsmoker occasional drinker Serious Relationship Marriage,0
17719,Zxxxx,26,Profesora,single,female,hispanic Straight Christian nonsmoker never Friendship Romance,0


In [286]:
# final 5583 rows
realdf.to_csv('real_profile.csv', header=True, index=False)

# combined dataset

In [319]:
final_df = pd.concat([realdf,scamdf1])


In [320]:
final_df.sample(10)

,username,age,occupation,status,gender,filtered sentence,y
13999,Punto73,49,Seller,single,male,white Straight Muslim nonsmoker never Friendship Serious Relationship,0
266,Armybeb,33,military,single,female,My name Victoria 34 year old single children never married I affectionate caring honest trustworthy lady simple personality Im Virginia I work US Army currently deployed SyriaMy deployment ends soon joined site hoping find man meet deployment honestserious relationship relationship something never got chance enjoy devotion commitment work I really funny person loves people loves laugh always love make happy Speaking hobbies I enjoy swimming going walk music movies sports Im looking forward seeing new places meeting new people new things Im fighter Im arguing Life short I value friendship intimacy commitment I enjoy food movies walks quiet times I love nature enjoy learning deep respect love children I fascinated world different cultures would like learn I love life well animals still love share special man,1
1033,Amanda18,45,Docente,separated,female,hispanic Straight Other nonsmoker never Friendship,0
16297,ThomasSantos93,29,Fedex,single,male,asian Straight Other nonsmoker never Fun Friendship,0
8905,Jswag959,39,Food service,divorced,male,white Straight Christian nonsmoker social drinker Fun Friendship Romance Serious Relationship,0
2608,Cabezas,32,Vendedor,single,male,hispanic Straight Other nonsmoker social drinker Fun Friendship Romance Serious Relationship Marriage,0
17066,Werdstrom69,42,Machinist,single,male,Very single Looking get close sexy lady similar tastes No games strings unless want em,0
3316,chris80422,39,Retail,single,male,white Straight Christian nonsmoker occasional drinker Fun,0
15470,shakil,42,business,single,male,asian Straight Muslim nonsmoker never Marriage,0
15817,Spaguetti0225,38,Almacenista,single,male,hispanic Straight Other nonsmoker occasional drinker Romance,0


In [321]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8543 entries, 26 to 4341
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   username           8543 non-null   object
 1   age                8543 non-null   object
 2   occupation         8543 non-null   object
 3   status             8543 non-null   object
 4   gender             8543 non-null   object
 5   filtered sentence  8543 non-null   object
 6   y                  8543 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 533.9+ KB


In [185]:
#read to csv file
final_df.to_csv('combined_profile.csv', header=True, index=False)

In [28]:
final_df[final_df["username"] == "Aigul0468"]

,username,age,occupation,status,gender,filtered sentence,y
0,Aigul0468,54,Социальная,divorced,female,asian Straight Muslim nonsmoker never Serious Relationship Marriage,0


# 4. Distribution plots for categorical variables 

## 4.1 Age distribution of real profiles

In [31]:
import matplotlib.pyplot as plt
import plotly.express as px

counts = realdf["Age"].value_counts(normalize = True)

fig = px.bar(counts, title="Age of real profiles")
fig.update_layout(
    xaxis_title = "Age",
    yaxis_title = "Number of profiles",
    title_x = 0.5,
    showlegend = False
)

fig.show()

In [40]:
import matplotlib.pyplot as plt
import plotly.express as px

counts = scamdf1["age"].value_counts(normalize = True)

fig = px.bar(counts, title="Age of scam profiles")
fig.update_layout(
    xaxis_title = "Age",
    yaxis_title = "Number of profiles",
    title_x = 0.5,
    showlegend = False
)

fig.show()

## 4.1 Gender distribution 

In [26]:
import matplotlib.pyplot as plt
import plotly.express as px

counts = realdf["gender"].value_counts(normalize = True)

fig = px.bar(counts, title="Gender distribution of real profiles")
fig.update_layout(
    xaxis_title = "Gender",
    yaxis_title = "Percentage of profiles",
    title_x = 0.5,
    showlegend = False
)

fig.show()

In [27]:
import matplotlib.pyplot as plt
import plotly.express as px

counts = scamdf["gender"].value_counts(normalize = True)

fig = px.bar(counts, title="Gender distribution of scam profiles")
fig.update_layout(
    xaxis_title = "Gender",
    yaxis_title = "Percentage of profiles",
    title_x = 0.5,
    showlegend = False
)

fig.show()

## 4.6 Status

In [28]:
counts = realdf["status"].value_counts(normalize = True)

fig = px.bar(counts, title="Status of real profiles")
fig.update_layout(
    xaxis_title = "Status",
    yaxis_title = "Percentage of real profiles",
    title_x = 0.5,
    showlegend = False
)

fig.show()

In [29]:
counts = scamdf["status"].value_counts(normalize = True)

fig = px.bar(counts, title="Status of scam profiles")
fig.update_layout(
    xaxis_title = "Status",
    yaxis_title = "Percentage of scam profiles",
    title_x = 0.5,
    showlegend = False
)

fig.show()

In [45]:
counts = scamdf["occupation"].value_counts(normalize = True)

fig = px.bar(counts, title="Occupation of scam profiles")
fig.update_layout(
    xaxis_title = "Occupations",
    yaxis_title = "Percentage of scam profiles",
    title_x = 0.5,
    showlegend = False
)

fig.show()

In [46]:
counts = realdf["occupation"].value_counts(normalize = True)

fig = px.bar(counts, title="Occupation of real profiles")
fig.update_layout(
    xaxis_title = "Occupations",
    yaxis_title = "Percentage of real profiles",
    title_x = 0.5,
    showlegend = False
)

fig.show()